In [1]:
import numpy as np
import string
import nltk
import random
from nltk.stem import WordNetLemmatizer
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from datetime import datetime, date
import tensorflow as tf

In [16]:
import json
stop_words = ['is','am','the','a','an','be','are','were',]


text = open('text.json').read()
data = json.loads(text)

In [17]:
lemmatizer = WordNetLemmatizer()
words = []
y = []
patterns =[]
x = []
classes = []
# getting the classes(tags) output and patterns from 'data'
for intent in data['intents']:
    classes.append(intent['tag'])
    for pattern in intent['patterns']:
# applying tokenizer to convert the sentences into a list of words        
        tokens = nltk.word_tokenize(pattern)
# here .extend() is used instead of append() since we don't want to append lists in words 
# but its elements i.e words in 'token'
# here 'tokens' is a list of words
        words.extend(tokens)
        patterns.append(pattern)
        y.append(intent['tag'])

# converting to lower case, applyging lemmatization removing the puctuations
# here 'words' is our vocabulary containing all the words 
words = [lemmatizer.lemmatize(word.lower()) for word  in words if word not in string.punctuation and 
        word not in stop_words]
# converting the list to set to avoid doubling of words in in 'words'
words = sorted(set(words))
words = list(words)

for list_ in patterns:
    list_ = nltk.word_tokenize(list_)
    list_ = [lemmatizer.lemmatize(lis.lower()) for lis in list_ if lis not in string.punctuation and lis not in stop_words]
    x.append(list_)
        

In [ ]:
print('classes\n',classes,'\n')
print('words\n',words,'\n')
print('x\n',x,'\n')
print('y\n',y,'\n')
len(words)

In [18]:
# applying the one hot encoding on our training data
train_x = []
train_y = []
training = []
empty_list = [0]*len(classes)
for idx,list_ in enumerate(x):
    doc = []
    for word in words:
        doc.append(1) if word in list_ else doc.append(0)
    output = list(empty_list)
    output[classes.index(y[idx])] = 1
    training.append([doc,output])
random.shuffle(training)       
training = np.array(training,dtype = object)
train_x = list(training[:,0])
train_x = np.array(train_x)
train_y = np.array(list(training[:,1]))


In [19]:
ACCURACY_THRESHOLD = 0.99
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            print("\nTraining stopped at epoch number:",epoch)   
            self.model.stop_training = True
callbacks = myCallback()        

In [20]:
input_shape = (len(train_x[1]),)
output_shape = len(train_y[0])      
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
model.fit(x=train_x, y=train_y, epochs=200, verbose=1,callbacks=[callbacks])


Epoch 1/200
3/3 [==============================] - 0s 3ms/step - loss: 2.6046 - accuracy: 0.1331
Epoch 2/200
3/3 [==============================] - 0s 3ms/step - loss: 2.4269 - accuracy: 0.2333
Epoch 3/200
3/3 [==============================] - 0s 3ms/step - loss: 2.2001 - accuracy: 0.3530
Epoch 4/200
3/3 [==============================] - 0s 4ms/step - loss: 2.0684 - accuracy: 0.3323
Epoch 5/200
3/3 [==============================] - 0s 3ms/step - loss: 1.7499 - accuracy: 0.4309
Epoch 6/200
3/3 [==============================] - 0s 2ms/step - loss: 1.4889 - accuracy: 0.6219
Epoch 7/200
3/3 [==============================] - 0s 3ms/step - loss: 1.2820 - accuracy: 0.6203
Epoch 8/200
3/3 [==============================] - 0s 4ms/step - loss: 0.9816 - accuracy: 0.7628
Epoch 9/200
3/3 [==============================] - 0s 3ms/step - loss: 0.8976 - accuracy: 0.7840
Epoch 10/200
3/3 [==============================] - 0s 3ms/step - loss: 0.7147 - accuracy: 0.7929
Epoch 11/200
3/3 [===========

In [23]:
model.save('ZBOT.h5')

In [21]:
def bagofwords(msg):
    tokens = nltk.word_tokenize(msg)
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token not in string.punctuation and token 
              not in stop_words]
    binary_msg = []
    for word in words:
        binary_msg.append(1) if word in tokens else binary_msg.append(0)
    return np.array(binary_msg)
def prediction(msg):
    message = bagofwords(msg)
    result = model.predict(np.array([message]))
    result = np.argmax(result)
    class_index = result
    for intent in data['intents']:
        if intent['tag'] == classes[class_index]:
            if classes[class_index] == 'time':
                now = datetime.now()
                current_time = now.strftime("%H:%M:%S")
                response = random.choice(intent['responses']) + current_time
                return response
            elif classes[class_index] == 'date':
                today = date.today()
                current_date = today.strftime("%d/%m/%Y")
                response = random.choice(intent['responses']) + current_date
                return response
            response = random.choice(intent['responses'])
            return response

In [22]:
# to get user input in  multiples lines
from tkinter import *
import tkinter as tk
window = tk.Tk()
# detemining how large widow should be and where it should be on  the screen
window.geometry("662x445+220+50")
# this is how to set background color of window
window['background']='white'
# giving title
window.title('ZBOT')
# function to get input from text box and to print that text on other text box
def getinput():
    # getting text
    text = entry.get("1.0", tk.END)
    response =prediction(text)
    # deleting what is written just now when after the button click
    entry.delete(1.0,tk.END)
    # insert the text on other text box at the end
    text_box.config(state = tk.NORMAL)    
    text_box.insert(tk.END,'YOU:   '+text)
    text_box.insert(tk.END,'ZBOT:   ' + response + '\n\n')
    text_box.yview(END)
    text_box.config(state = tk.DISABLED)

text_box = tk.Text(window, bg ="#856ff8", fg = 'white',font=("Verdana",12,'bold')
                  ,height =21,width =58 )
text_box.config(state = tk.DISABLED)
scrollbar = tk.Scrollbar(window, command=text_box.yview)
scrollbar.place(x=645,y=6, height= 435)
# .place() is a geometry manager used to set size of widgets like button, entry box 
text_box.place(x =2,y = 3)
button = tk.Button(window,text ='ENTER', width = 17,height =2,bg ="blue",
                   fg ='white',font=("Verdana",12,'bold'), command = getinput)
button.place(x=450, y=390)
entry = tk.Text(window,height =2,width =40,bg ='yellow',fg = 'blue',font='bold')
entry.place(x = 1, y = 390)
# used to display window
window.mainloop()